# Importamos librerias necesarias

In [1]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import re
from GoalData import Goal
import pandas as pd

### Realizamos nuestra peticion para obtener la data

In [2]:
url = "https://www.messivsronaldo.app/match-histories/ronaldo-match-history/"

try:
    response = requests.get(url)
    response.raise_for_status()
    html_content = response.text
except requests.exceptions.RequestException as e:
    print(f"Error al obtener la página: {e}")
    html_content = None

### Parseamos nuestro documento

In [3]:
tree = html.fromstring(html_content)

### Realizamos nuestra consulta de datos con xPath

In [4]:
fechas = tree.xpath('//div[@class="mb-3 text-sm text-slate-200 relative lg:flex lg:flex-wrap"]/div[@class="MatchHistory-module--metaWrap--aba5a flex lg:flex-wrap lg:content-start text-xs w-full lg:w-2/5 lg:order-1 text-slate-400/75 py-1 px-2 rounded-t gap-x-2 row-gap-1"]/span[1]')

In [5]:
competiciones = tree.xpath('//div[@class="mb-3 text-sm text-slate-200 relative lg:flex lg:flex-wrap"]/div[@class="MatchHistory-module--metaWrap--aba5a flex lg:flex-wrap lg:content-start text-xs w-full lg:w-2/5 lg:order-1 text-slate-400/75 py-1 px-2 rounded-t gap-x-2 row-gap-1"]/span[2]')

In [6]:
visita_local = tree.xpath('//div[@class="mb-3 text-sm text-slate-200 relative lg:flex lg:flex-wrap"]/div[@class="MatchHistory-module--metaWrap--aba5a flex lg:flex-wrap lg:content-start text-xs w-full lg:w-2/5 lg:order-1 text-slate-400/75 py-1 px-2 rounded-t gap-x-2 row-gap-1"]/span[contains(., "(H)") or contains(., "(A)") or contains(., "(N)")]')

In [7]:
partidos = tree.xpath('//div[@class="mb-3 text-sm text-slate-200 relative lg:flex lg:flex-wrap"]/div/button/div[@class="flex flex-wrap text-sm w-2/5 leading-tight"]')

Usamos RegEx para separar marcadores, ya que no se encontró un patron en los tags para obtenerlos por separado 

In [8]:
print(len(partidos))
re_partidos = list()
for partido in partidos:
    re_partidos.append(re.split(r'(?=\d{1,})(?<!1)|(?=[A-Za-z])(?<=\d)', partido.text_content()))

1281


In [9]:
goles = tree.xpath('//div[contains(@title, "Goals")]')

In [10]:
asistencias = tree.xpath('//div[contains(@title, "Assists")]')

In [11]:
minutos_jugados = tree.xpath('//span[contains(@title, "minutes")]')

### Creamos un array de Goles de la clase Goal

In [12]:
goals = list()
for i in range(len(fechas)):
    goals.append(Goal(fechas[i].text_content(),
                    competicion = competiciones[i].text_content(),
                    visita_local = visita_local[i].text_content().replace(')','').replace('(',''),
                    equipo_local = re_partidos[i][0],
                    equipo_visitante = re_partidos[i][2],
                    marcador_local = re_partidos[i][1],
                    marcador_visitante = re_partidos[i][3],
                    goles_anotados = goles[i].text_content(),
                    asistencias = asistencias[i].text_content(),
                    minutos_jugados = minutos_jugados[i].text_content()))

### Creamos DataFrame de Pandas

In [13]:
goles = [goal.to_dict() for goal in goals]

In [14]:
goles_df = pd.DataFrame(goles)

In [15]:
goles_df.head(5)

,dia,competicion,visita_local,equipo_local,equipo_visitante,marcador_local,marcador_visitante,goles_anotados,asistencias,minutos_jugados
0,08/06/2025,Nations League,N,Portugal,Spain,2,2,1,0,88' mins played
1,04/06/2025,Nations League,A,Germany,Portugal,1,2,1,0,89' mins played
2,26/05/2025,Saudi Pro League,A,Al Fateh,Al Nassr,3,2,1,0,90' mins played
3,21/05/2025,Saudi Pro League,H,Al Nassr,Al-Khaleej,2,0,1,0,90' mins played
4,07/05/2025,Saudi Pro League,H,Al Nassr,Al-Ittihad,2,3,0,0,90' mins played


In [16]:
goles_df[goles_df['visita_local'] == 'N'].groupby('equipo_local')['equipo_local'].count()

equipo_local
Al Nassr             11
Juventus              5
Manchester United    13
Portugal             57
Real Madrid          13
Name: equipo_local, dtype: int64

In [17]:
goles_df['minutos_jugados'] = goles_df['minutos_jugados'].str.extract(r'(\d+)')

In [18]:
goles_df['marcador_local'] = goles_df['marcador_local'].astype(int)
goles_df['marcador_visitante'] = goles_df['marcador_visitante'].astype(int)
goles_df['goles_anotados'] = goles_df['goles_anotados'].astype(int)
goles_df['asistencias'] = goles_df['asistencias'].astype(int)
goles_df['minutos_jugados'] = goles_df['minutos_jugados'].astype(int)
# goles_df['dia'] = pd.to_datetime(goles_df['dia'])

In [19]:
goles_df.head(5)

,dia,competicion,visita_local,equipo_local,equipo_visitante,marcador_local,marcador_visitante,goles_anotados,asistencias,minutos_jugados
0,08/06/2025,Nations League,N,Portugal,Spain,2,2,1,0,88
1,04/06/2025,Nations League,A,Germany,Portugal,1,2,1,0,89
2,26/05/2025,Saudi Pro League,A,Al Fateh,Al Nassr,3,2,1,0,90
3,21/05/2025,Saudi Pro League,H,Al Nassr,Al-Khaleej,2,0,1,0,90
4,07/05/2025,Saudi Pro League,H,Al Nassr,Al-Ittihad,2,3,0,0,90


In [20]:
goles_df.to_csv('./data/cr7Matches.csv')